1. yfinance, matplotlib, pandas, seaborn 라이브러리 설치하기

In [ ]:
pip install yfinance matplotlib seaborn

2. 주식 데이터 분석에 따른 다양한 시각화하기

<u>**주식 데이터 가져오기**</u>

In [ ]:
import yfinance as yf
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# 1 주식 데이터 가져오기
ticker = "AAPL" # 애플 티커
data = yf.download(ticker, period="5y")
data.columns = ['고가', '저가', '종가', '조정 종가', '거래량', '시가']

# 2 한글 폰트 설정
plt.rc('font', family='Gulim') # 'AppleGothic' (맥 사용 시)

In [ ]:
data

<u>**주가 시계열 그래프 그리기**</u>

In [ ]:
# 주가 시계열 그래프
plt.figure(figsize=(12, 6)) # 1 그래프 크기를 설정
plt.plot(data.index, data['조정 종가'], label='조정 종가') # 2 선 그래프 그리기
plt.title(f'{ticker} 주가 시계열 그래프') # 3 그래프 제목 설정
plt.xlabel('날짜') # 4 x축 라벨 설정
plt.ylabel('조정 종가') # 5 y축 라벨 설정
plt.legend() # 6 범례 추가
plt.show() # 7 그래프 표시

<u>**이동 평균선을 추가한 시계열 그래프 그리기**</u>

In [ ]:
# 이동 평균선을 추가한 시계열 그래프
# 1 이동 평균을 계산하여 새로운 열로 추가
data['50일 이동 평균'] = data['조정 종가'].rolling(window=50).mean()
data['200일 이동 평균'] = data['조정 종가'].rolling(window=200).mean()

# 2 그래프 그리기
plt.figure(figsize=(12, 6)) # 그래프 크기를 설정
plt.plot(data.index, data['조정 종가'], label='조정 종가') # 종가 선 그래프 그리기
plt.plot(data.index, data['50일 이동 평균'], label='50일 이동 평균') # 50일 이동 평균선 그래프 그리기
plt.plot(data.index, data['200일 이동 평균'], label='200일 이동 평균') # 200일 이동 평균선 그래프 그리기
plt.title(f'{ticker} 주가 및 이동 평균선') # 그래프 제목 설정
plt.xlabel('날짜') # x축 라벨 설정
plt.ylabel('가격') # y축 라벨 설정
plt.legend() # 범례 추가
plt.show() # 그래프 표시

<u>**볼린저 밴드를 추가한 시계열 그래프 그리기**</u>

In [ ]:
# 볼린저 밴드를 추가한 시계열 그래프
# 1 필요한 데이터 계산하기
data['20일 이동 평균'] = data['조정 종가'].rolling(window=20).mean() # 20일 이동 평균
data['20일 표준편차'] = data['조정 종가'].rolling(window=20).std() # 20일 표준편차
data['상단 밴드'] = data['20일 이동 평균'] + (data['20일 표준편차'] * 2) # 상단 밴드
data['하단 밴드'] = data['20일 이동 평균'] - (data['20일 표준편차'] * 2) # 하단 밴드

# 2 그래프 그리기
plt.figure(figsize=(12, 6)) # 그래프 크기를 설정
plt.plot(data.index, data['조정 종가'], label='조정 종가') # 종가 선 그래프 그리기
# 이동 평균 선 그래프 그리기
plt.plot(data.index, data['20일 이동 평균'], label='20일 이동 평균')
# 상단 밴드 선 그래프 그리기
plt.plot(data.index, data['상단 밴드'], label='상단 밴드')
# 하단 밴드 선 그래프 그리기
plt.plot(data.index, data['하단 밴드'], label='하단 밴드')
plt.fill_between(data.index, data['하단 밴드'], data['상단 밴드'], color='gray',
alpha=0.3) # 상단 밴드와 하단 밴드 사이 영역을 회색으로 채우기
plt.title(f'{ticker} 주가 및 볼린저 밴드') # 그래프 제목 설정
plt.xlabel('날짜') # x축 라벨 설정
plt.ylabel('가격') # y축 라벨 설정
plt.legend() # 범례 추가
plt.show() # 그래프 표시

<u>**일별 수익률 시계열 그래프 그리기**</u>

In [ ]:
# 일별 수익률 시계열 그래프
# 1 일별 수익률 계산하기
data['일별 수익률'] = data['조정 종가'].pct_change()

# 2 그래프 그리기
plt.figure(figsize=(12, 6)) # 그래프 크기를 설정
plt.plot(data.index, data['일별 수익률'], label='일별 수익률') # 일별 수익률 선 그래프 그리기
plt.title(f'{ticker} 일별 수익률 시계열 그래프') # 그래프 제목 설정
plt.xlabel('날짜') # x축 라벨 설정
plt.ylabel('일별 수익률') # y축 라벨 설정
plt.legend() # 범례 추가
plt.show() # 그래프 표시

<u>**여러 주식의 상관관계 행렬 그래프 그리기**</u>

In [ ]:
# 여러 주식의 상관관계 행렬 그래프
# 1 분석할 주식의 티커 목록 설정
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
# 2 각 주식의 5년간 조정 종가 데이터 다운로드
data_multi = yf.download(tickers, period="5y")['Adj Close']

# 3 주식들의 일간 수익률을 계산하고, 결측치(NaN) 제거
returns = data_multi.pct_change().dropna()
# 4 일간 수익률 간의 상관관계 행렬 계산
correlation_matrix = returns.corr()

# 5 그래프 그리기
plt.figure(figsize=(10, 8)) # 그래프 크기 설정
# 6 상관관계 행렬을 히트맵으로 표시하고, 각 셀에 상관계수 표시
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('주식 가격 상관관계 행렬') # 그래프 제목 설정
plt.show() # 그래프 표시